<a href="https://colab.research.google.com/github/Jack3690/Kugelblitz/blob/main/Bar_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Initialization**

## **SDSS-Marvin**

In [1]:
!pip install sdss-marvin --quiet

     |████████████████████████████████| 393 kB 5.6 MB/s 
     |████████████████████████████████| 50 kB 4.2 MB/s 
     |████████████████████████████████| 284 kB 26.7 MB/s 
     |████████████████████████████████| 54 kB 1.4 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
     |████████████████████████████████| 525 kB 36.7 MB/s 
     |████████████████████████████████| 596 kB 40.9 MB/s 
     |████████████████████████████████| 169 kB 46.1 MB/s 
     |████████████████████████████████| 100 kB 8.6 MB/s 
     |████████████████████████████████| 548 kB 47.2 MB/s 
     |████████████████████████████████| 212 kB 44.2 MB/s 
     |████████████████████████████████| 40 kB 5.0 MB/s 
     |████████████████████████████████| 41 kB 323 kB/s 
     |████████████████████████████████| 111 kB 46.2 MB/s 
     |████████████████████████████████| 47 kB 4.7 MB/s 
     |██████████████████████████

In [2]:
!git clone https://github.com/Jack3690/Kugelblitz

Cloning into 'Kugelblitz'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 38 (delta 6), reused 31 (delta 2), pack-reused 0
Unpacking objects: 100% (38/38), done.


### **Configuring Marvin**

In [3]:
cd Kugelblitz/

/content/Kugelblitz


In [10]:
!cp -a sas/ /root/

!cp -a sas/map.py /usr/local/lib/python3.7/dist-packages/marvin/utils/plot/

!mkdir  /root/sas/dr16
!cp -a /root/sas/mangawork/manga/ /root/sas/dr16

mkdir: cannot create directory ‘/root/sas/dr16’: File exists


In [11]:
import os
import sys
import requests
import warnings
import PIL
import copy

from matplotlib import pyplot as plt
from matplotlib import colors
import seaborn as sb
import matplotlib
import numpy as np
from matplotlib import gridspec
import pandas as pd
import marvin.utils.plot.colorbar as colorbar
from matplotlib.colors import ListedColormap
from IPython.display import clear_output

from marvin.tools.maps import Maps
import marvin.utils.plot.map as mapplot
from astropy.table import Table
import astropy.units as u
from astropy.io import ascii
from astropy.wcs import WCS
from astropy.coordinates import Distance
from astropy.cosmology import WMAP9 as cosmo
from astropy.cosmology import Planck15
from astropy.io import fits
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
if sys.version_info.major == 2:
    from cStringIO import StringIO as stringio
else:
    from io import StringIO as stringio
    from io import BytesIO as bytesio

matplotlib.rcParams['figure.figsize']=(10,10)
matplotlib.rcParams['font.size']=15

#sb.set_style('dark')

/usr/local/lib/python3.7/dist-packages/marvin/core/exceptions.py:51: UserWarning: cannot initiate Sentry error reporting: [Errno 6] No such device or address.
  UserWarning)
[INFO]: No release version set. Setting default to DR15


## **Fast.ai**

In [12]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 5.4 MB/s 
     |████████████████████████████████| 1.2 MB 37.3 MB/s 
     |████████████████████████████████| 46 kB 3.8 MB/s 
     |████████████████████████████████| 186 kB 52.6 MB/s 
     |████████████████████████████████| 56 kB 3.4 MB/s 
     |████████████████████████████████| 51 kB 272 kB/s 
Mounted at /content/gdrive


In [13]:
from fastai.vision.all import *
from fastbook import *
from fastai.vision.widgets import *

In [14]:
path=Path('/content/gdrive/My Drive')
path.exists()

True

# **Data & Image acquisition**

In [15]:
import matplotlib.pyplot as plt
import pandas as pd

**Table with MaNGA Galaxies**

In [17]:
df=pd.read_csv('data/mangaifu.csv')
df=df[df.ifudesignsize>0][["plateifu","mangaid","objra","objdec"]]
df

,plateifu,mangaid,objra,objdec
0,8935-12701,1-404249,194.523424,29.017353
1,7977-12701,1-114955,332.602090,11.713077
2,9494-12701,1-412866,126.910630,21.256855
3,8150-12701,1-389685,146.701430,31.719579
4,9487-12701,1-44618,122.522873,46.193025
...,...,...,...,...
4819,8155-9102,1-202101,53.686515,-1.245784
4820,9042-9102,1-549071,235.032077,28.152265
4821,9870-9102,1-199224,233.165629,43.057807
4822,8719-9102,1-44693,121.744779,46.562969


In [ ]:
if not os.path.exists(path/'mangaifu/'):
  os.mkdir(path/'mangaifu/')
count = 0
for plateifu in df.plateifu:
  count+=1
  try:
    maps = Maps(plateifu=plateifu,mode='remote')

    sdss_cutout = maps.getImage()
    scale = maps.wcs.to_header()['PC2_2']*3600
    size = 2*maps.wcs.to_header()['CRPIX1']*scale
    sdss_cutout.get_new_cutout(size,size,scale=size/512)

    sdss_cutout.save(path/f'mangaifu/{plateifu}.png')

    if count%10==0:
      print('.',end='')
    if count%1000==0:
      print(count)
  except:
    print(f"Error for plateifu : {plateifu}")

Error for plateifu : 8480-12701
.

In [ ]:
sdss_cutout.save(path/'mangaifu/plateifu.png')

# **Caffe Classifier (Sheelu et.al)**

https://doi.org/10.1093/mnras/sty627

In [9]:
!git clone https://github.com/vadimkantorov/caffemodel2pytorch

Cloning into 'caffemodel2pytorch'...
remote: Enumerating objects: 294, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 294 (delta 1), reused 0 (delta 0), pack-reused 287
Receiving objects: 100% (294/294), 81.26 KiB | 2.71 MiB/s, done.
Resolving deltas: 100% (98/98), done.


In [10]:
import caffemodel2pytorch.caffemodel2pytorch as caffemodel2pytorch

In [11]:
model = caffemodel2pytorch.Net(
	prototxt = path/'bar_model/deploy.prototxt',
	weights = path/'bar_model/snapshot_iter_471780.caffemodel',
	caffe_proto = 'https://raw.githubusercontent.com/BVLC/caffe/master/src/caffe/proto/caffe.proto'
);

caffemodel2pytorch: loading model from [/content/gdrive/My Drive/bar_model/snapshot_iter_471780.caffemodel] in HDF5 format failed [Unable to open file (file signature not found)], falling back to caffemodel format
caffemodel2pytorch: loaded model from [/content/gdrive/My Drive/bar_model/snapshot_iter_471780.caffemodel] in caffemodel format


In [ ]:
image_path=get_image_files(path/'WHAN/')

In [ ]:
# Dict object to store data
barclass= {
    'barred' : [],
    'unbarred': [],
}
count=0
for path_img in image_path:
  # Loading image from file
  img=Image.open(path_img)
  image=img.resize((256,256))
  data=tensor(image).permute(2,1,0).unsqueeze(0).float()
  if torch.cuda.is_available():
    data=data.cuda()
  
  # Clasifying using Caffe model
  out= model(data = data)

  # Using output probabilities to find class
  if ( (out['softmax'][0][0].item() > out['softmax'][0][1].item()) and (out['softmax'][0][0].item() > 0.5) ):
    ch='barred'
    prob=out['softmax'][0][0].item() 
  elif ( (out['softmax'][0][0].item() < out['softmax'][0][1].item()) and (out['softmax'][0][1].item() > 0.5) ):
    ch='unbarred'
    prob = out['softmax'][0][1].item()

  # Appending data to dict
  barclass[ch].append([path_img.name[:-4],prob])
  count+=1

  print(".",end="")
  if count%100==0:
    print(count)
print(count)

....................................................................................................100
....................................................................................................200
....................................................................................................300
....................................................................................................400
....................................................................................................500
....................................................................................................600
....................................................................................................700
....................................................................................................800
....................................................................................................900
................................................................

In [ ]:
bar=pd.DataFrame(barclass['barred'],columns=['plateifu','prob'])
nobar=pd.DataFrame(barclass['unbarred'],columns=['plateifu','prob'])

In [ ]:
bar.to_csv(path/'manga_caffe_bar.csv')
nobar.to_csv(path/'manga_caffe_unbarred.csv')

In [ ]:
print("Number of barred galaxies :",bar.count()[0])
print("Number of unbarred galaxies :",nobar.count()[0])
print("Total :",bar.count()[0]+nobar.count()[0])

Number of barred galaxies : 77
Number of unbarred galaxies : 4713
Total : 4790


# **ResNet152 Classifier (Fast.ai)**
Work in progess

In [ ]:
learn = load_learner(path/'bar_model/export.pkl')

In [ ]:
learn.dls.vocab

['barred', 'unbarred']

In [ ]:
(path/'WHAN/').exists()

True

In [ ]:
image_path=get_image_files(path/'WHAN/')

In [ ]:
len(image_path)

4790

In [ ]:
# Dict object to store data
barclass= {
    'barred' : [],
    'unbarred': [],
}
count=0
for img in image_path:
  # Loading and classifying image
  c,i,p=learn.predict(img)
  # Appening data to dict 
  barclass[c].append([img.name[:-4],p[i].numpy().item()])
  count+=1
  if count%100==0:
    print(count)
    break
print(count)

100
100


In [ ]:
bar=pd.DataFrame(barclass['barred'],columns=['plateifu','prob'])
nobar=pd.DataFrame(barclass['unbarred'],columns=['plateifu','prob'])

In [ ]:
bar.to_csv(path/'manga_bar.csv')
nobar.to_csv(path/'manga_unbarred.csv')

In [ ]:
print("Number of barred galaxies :",bar.count()[0])
print("Number of unbarred galaxies :",nobar.count()[0])
print("Total :",bar.count()[0]+nobar.count()[0])

Number of barred galaxies : 1784
Number of unbarred galaxies : 3006
Total : 4790
